## Harjoitustyö

## Vaihe 1

In [16]:
# Kirjastot

import pandas as pd
import numpy as np
import requests
import json
import xml.etree.ElementTree as e
import sqlite3
from sqlalchemy import create_engine
import re

#### Tehtävä 1. Hae Digitrafficin REST-rajapintapalvelusta (JSON-formaatissa) aktiiviset junat pääteosoitteella `/live-trains` Helsingin asemalta, joiden hakuparametrit ovat seuraavat:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [2]:
url = "https://rata.digitraffic.fi/api/v1/live-trains/station/HKI/"
params = {
    "arrived_trains": 10,
    "departed_trains": 10,
    "include_nonstopping": False
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    for train in data:
        print(train)
else:
    print(f"Virhekoodi: {response.status_code}, Virheviesti: {response.text}")

data

{'trainNumber': 3, 'departureDate': '2023-09-15', 'operatorUICCode': 10, 'operatorShortCode': 'vr', 'trainType': 'IC', 'trainCategory': 'Long-distance', 'commuterLineID': '', 'runningCurrently': False, 'cancelled': False, 'version': 286477974655, 'timetableType': 'REGULAR', 'timetableAcceptanceDate': '2023-07-05T11:22:01.000Z', 'timeTableRows': [{'stationShortCode': 'HKI', 'stationUICCode': 1, 'countryCode': 'FI', 'type': 'DEPARTURE', 'trainStopping': True, 'commercialStop': True, 'commercialTrack': '7', 'cancelled': False, 'scheduledTime': '2023-09-15T07:15:00.000Z', 'causes': [], 'trainNumber': 3, 'trainReady': {'source': 'KUPLA', 'accepted': True, 'timestamp': '2023-09-15T07:10:17.000Z'}}, {'stationShortCode': 'PSL', 'stationUICCode': 10, 'countryCode': 'FI', 'type': 'ARRIVAL', 'trainStopping': True, 'commercialStop': True, 'commercialTrack': '4', 'cancelled': False, 'scheduledTime': '2023-09-15T07:20:00.000Z', 'causes': [], 'trainNumber': 3}, {'stationShortCode': 'PSL', 'stationUIC

[{'trainNumber': 3,
  'departureDate': '2023-09-15',
  'operatorUICCode': 10,
  'operatorShortCode': 'vr',
  'trainType': 'IC',
  'trainCategory': 'Long-distance',
  'commuterLineID': '',
  'runningCurrently': False,
  'cancelled': False,
  'version': 286477974655,
  'timetableType': 'REGULAR',
  'timetableAcceptanceDate': '2023-07-05T11:22:01.000Z',
  'timeTableRows': [{'stationShortCode': 'HKI',
    'stationUICCode': 1,
    'countryCode': 'FI',
    'type': 'DEPARTURE',
    'trainStopping': True,
    'commercialStop': True,
    'commercialTrack': '7',
    'cancelled': False,
    'scheduledTime': '2023-09-15T07:15:00.000Z',
    'causes': [],
    'trainNumber': 3,
    'trainReady': {'source': 'KUPLA',
     'accepted': True,
     'timestamp': '2023-09-15T07:10:17.000Z'}},
   {'stationShortCode': 'PSL',
    'stationUICCode': 10,
    'countryCode': 'FI',
    'type': 'ARRIVAL',
    'trainStopping': True,
    'commercialStop': True,
    'commercialTrack': '4',
    'cancelled': False,
    'sc

#### Tehtävä 2. Tämän jälkeen lue kyseinen data Pandasin dataframeen ilman muokkaamista, jolloin sarakkeiden otsikot ovat seuraavanlaiset:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version` 

In [51]:
junat = pd.DataFrame(data)

junat.columns = [
    "cancelled",
    "commuterLineID",
    "departureDate",
    "operatorShortCode",
    "operatorUICCode",
    "runningCurrently",
    "timeTableRows",
    "timetableAcceptanceDate",
    "timetableType",
    "trainCategory",
    "trainNumber",
    "trainType",
    "version"
]

df = pd.DataFrame(junat)

df.head(4)

,cancelled,commuterLineID,departureDate,operatorShortCode,operatorUICCode,runningCurrently,timeTableRows,timetableAcceptanceDate,timetableType,trainCategory,trainNumber,trainType,version
0,3,2023-09-15,10,vr,IC,Long-distance,,False,False,286477974655,REGULAR,2023-07-05T11:22:01.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
1,33,2023-09-15,10,vr,S,Long-distance,,True,False,286477982388,REGULAR,2023-07-05T11:22:01.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
2,140,2023-09-15,10,vr,S,Long-distance,,False,False,286477886646,REGULAR,2023-07-05T11:22:01.000Z,"[{'stationShortCode': 'JY', 'stationUICCode': ..."
3,143,2023-09-15,10,vr,IC,Long-distance,,False,False,286477857178,REGULAR,2023-07-05T11:22:01.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


#### Tehtävä 3. Poista kaikki muut sarakkeet paitsi `timeTableRows` ja `trainNumber`

In [52]:
new_df = df[['timeTableRows', 'trainNumber']]

new_df

,timeTableRows,trainNumber
0,,REGULAR
1,,REGULAR
2,,REGULAR
3,,REGULAR
4,,REGULAR
5,A,REGULAR
6,A,REGULAR
7,E,REGULAR
8,E,REGULAR
9,U,REGULAR


#### Tehtävä 4. Tämän jälkeen pura `timeTableRows` sarakkeen arvot jokaisesta junasta erikseen (sarakkeen arvo jokaiselle junalle sisältää useampia JSON-objekteja) ja lisää dataan mukaan junan tiedot yksilöivä `trainNumber`

In [53]:
junat = {
    "cancelled": [],
    "commuterLineID": [],
    "departureDate": [],
    "operatorShortCode": [],
    "operatorUICCode": [],
    "runningCurrently": [],
    "timeTableRows": [],
    "timetableAcceptanceDate": [],
    "timetableType": [],
    "trainCategory": [],
    "trainNumber": [],
    "trainType": [],
    "version": []
}

for train in data:
    junat["cancelled"].append(train["cancelled"])
    junat["commuterLineID"].append(train["commuterLineID"])
    junat["departureDate"].append(train["departureDate"])
    junat["operatorShortCode"].append(train["operatorShortCode"])
    junat["operatorUICCode"].append(train["operatorUICCode"])
    junat["runningCurrently"].append(train["runningCurrently"])
    junat["timeTableRows"].append(train["timeTableRows"])
    junat["timetableAcceptanceDate"].append(train["timetableAcceptanceDate"])
    junat["timetableType"].append(train["timetableType"])
    junat["trainCategory"].append(train["trainCategory"])
    junat["trainNumber"].append(train["trainNumber"])
    junat["trainType"].append(train["trainType"])
    junat["version"].append(train["version"])
    
df = pd.DataFrame(junat)

new_df = df[['timeTableRows', 'trainNumber']]

new_df = new_df.explode('timeTableRows', ignore_index=True)

tTr_tN_df = pd.DataFrame(new_df)

uusi_df.head(5)

,timeTableRows,trainNumber
0,"{'stationShortCode': 'HKI', 'stationUICCode': ...",3
1,"{'stationShortCode': 'PSL', 'stationUICCode': ...",3
2,"{'stationShortCode': 'PSL', 'stationUICCode': ...",3
3,"{'stationShortCode': 'KÄP', 'stationUICCode': ...",3
4,"{'stationShortCode': 'KÄP', 'stationUICCode': ...",3


#### Tehtävä 5. Pudota sitten seuraavat sarakkeet datasta: `causes`, `differenceInMinutes` ja `commercialStop`

In [ ]:
junat

teht5_df = junat.copy()

pudotettavat_sarakkeet = ['causes', 'differenceInMinutes', 'commercialStop']

lopullinen_df = teht5_df.drop(columns=pudotettavat_sarakkeet)

lopullinen_df

#### Tehtävä 6. Täytä jokaisen rivin sarakkeen `actualTime` puuttuvat arvot `scheduledTime` -sarakkeen arvolla.

In [5]:
#

## Vaihe 2

#### Tehtävä 7. Pura tämän kansion sisällä olevan `data.html` -tiedoston `table`-elementin solujen arvot käyttäen regex-syntaksia ja yhdistä ne vaiheessa 1 luotuun dataframeen. Huom: tehtävässä tulee käyttää vain regexiä (Esim. Pandasin read_html()-metodin käyttö on kielletty)!

In [15]:
with open('data.html', 'r', encoding='utf-8') as file:
    data = file.read()

table_pattern = r'<table[^>]*>.*?</table>'
cell_pattern = r'<td[^>]*>(.*?)</td>'

table_matches = re.findall(table_pattern, data, re.DOTALL)

df = pd.DataFrame()

for table_match in table_matches:
    cell_matches = re.findall(cell_pattern, table_match, re.DOTALL)
    df = pd.concat([df, pd.DataFrame([cell_matches])], ignore_index=True)

for col in df.columns:
    df[col] = df[col].str.strip()

df = df.values.reshape(3, 10)

columns = [
    "actualTime", "cancelled", "commercialTrack", "countryCode", "scheduledTime",
    "stationShortCode", "stationUICCode", "trainStopping", "type", "trainNumber"
]

df = pd.DataFrame(df, columns=columns)

df


,actualTime,cancelled,commercialTrack,countryCode,scheduledTime,stationShortCode,stationUICCode,trainStopping,type,trainNumber
0,2019-08-06T04:48:00.000Z,False,,FI,2019-08-06T04:48:00.000Z,JVS,1272,False,DEPARTURE,150
1,2019-08-06T06:04:39.000Z,False,,FI,2019-08-06T06:04:39.000Z,R702,98998,False,ARRIVAL,150
2,2019-08-06T07:03:12.000Z,False,,FI,2019-08-06T07:03:12.000Z,ILA,98998,9,ARRIVAL,8337


## Vaihe 3

#### Tehtävä 8. Tee SQLite tietokanta ja lisää sinne vaiheen 2 tuloksena saatu dataframe.


In [23]:
yhteys = sqlite3.connect('vaihe_3')

df.to_sql('vaihe_3_sqlite', yhteys, if_exists='replace', index=False)


cursor = yhteys.cursor()

cursor.execute('SELECT * FROM vaihe_3_sqlite')
rows = cursor.fetchall()
for row in rows:
    print(row)

yhteys.close()

('2019-08-06T04:48:00.000Z', 'False', '', 'FI', '2019-08-06T04:48:00.000Z', 'JVS', '1272', 'False', 'DEPARTURE', '150')
('2019-08-06T06:04:39.000Z', 'False', '', 'FI', '2019-08-06T06:04:39.000Z', 'R702', '98998', 'False', 'ARRIVAL', '150')
('2019-08-06T07:03:12.000Z', 'False', '', 'FI', '2019-08-06T07:03:12.000Z', 'ILA', '98998', '9', 'ARRIVAL', '8337')
